In [1]:
import pandas as pd
import os
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.learning_curve import learning_curve
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

/Users/Sam/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/Sam/anaconda2/lib/python2.7/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [2]:
os.chdir('/Users/Sam/ds/metis/metisgh/sf16_ds4/local/Project3')

In [3]:
df = pd.read_csv('default_of_credit_card_clients.csv')
df = df.dropna()

In [4]:
df = df.drop('ID', axis = 1)

In [5]:
df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,20000.0,2.0,2.0,1.0,24.0,2.0,2.0,-1.0,-1.0,-2.0,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1.0
1,120000.0,2.0,2.0,2.0,26.0,-1.0,2.0,0.0,0.0,0.0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1.0
2,90000.0,2.0,2.0,2.0,34.0,0.0,0.0,0.0,0.0,0.0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0.0
3,50000.0,2.0,2.0,1.0,37.0,0.0,0.0,0.0,0.0,0.0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0.0
4,50000.0,1.0,2.0,1.0,57.0,-1.0,0.0,-1.0,0.0,0.0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0.0


In [6]:
df['default payment next month'] = df['default payment next month'].replace(to_replace=0, value="Paid")
df['default payment next month'] = df['default payment next month'].replace(to_replace=1, value="Default")
target_names = ["Default", "Paid"]

In [7]:
X = df.drop('default payment next month', axis = 1)
target = df['default payment next month']

In [8]:
X = df[['LIMIT_BAL', 'SEX', 'EDUCATION','MARRIAGE','AGE']]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size = 0.3, random_state = 42)

In [10]:
modelknn = KNeighborsClassifier()
knnfit = modelknn.fit(X_train, y_train)
knny_pred = knnfit.predict(X_test)

In [11]:
accuracy_score(y_test, knny_pred)

0.74277777777777776

# Feature Scaling Limit Balance

In [12]:
df['LIMIT_BAL'] = df['LIMIT_BAL']/1000

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size = 0.3, random_state = 42)

In [14]:
X.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE
0,20000.0,2.0,2.0,1.0,24.0
1,120000.0,2.0,2.0,2.0,26.0
2,90000.0,2.0,2.0,2.0,34.0
3,50000.0,2.0,2.0,1.0,37.0
4,50000.0,1.0,2.0,1.0,57.0


In [15]:
modellist = [KNeighborsClassifier(), LogisticRegression(), GaussianNB(), DecisionTreeClassifier(), RandomForestClassifier()]

In [22]:
for model in modellist:
    modelfit = model.fit(X_train, y_train)
    modelypred = modelfit.predict(X_test)
    print str(model).split("(")[0] + ": " + str(round(accuracy_score(y_test, modelypred),3))
    

KNeighborsClassifier: 0.743
LogisticRegression: 0.782
GaussianNB: 0.782
DecisionTreeClassifier: 0.693
RandomForestClassifier: 0.724


In [ ]:
accuracy_score()

# Feature Selection

In [32]:
def onefeature(model, name):
    arr = []
    X = df[[name]]
    X_train, X_test, y_train, y_test = train_test_split(X, target, test_size = 0.3, random_state = 42)
    modelfit = model.fit(X_train, y_train)
    modely_pred = modelfit.predict(X_test)
    arr.append(round(accuracy_score(y_test, modely_pred),3))
    arr.append(round(recall_score(y_test, modely_pred, labels = ["Paid", "Default"], pos_label= "Default"),3))
    arr.append(round(precision_score(y_test, modely_pred, labels = ["Paid", "Default"], pos_label = "Default"),3))
    return arr

In [33]:
colnames = df.columns.tolist()
colnames.pop()

'default payment next month'

In [34]:
for title in colnames:
    print title
    print onefeature(KNeighborsClassifier(), title)

LIMIT_BAL
[0.759, 0.105, 0.334]
SEX
[0.782, 0.0, 0.0]
EDUCATION
[0.782, 0.0, 0.0]
MARRIAGE
[0.782, 0.0, 0.0]
AGE
[0.746, 0.07, 0.228]
PAY_0
[0.782, 0.009, 0.45]
PAY_2
[0.79, 0.32, 0.531]
PAY_3
[0.437, 0.736, 0.241]
PAY_4
[0.786, 0.265, 0.515]
PAY_5
[0.792, 0.249, 0.55]
PAY_6
[0.787, 0.238, 0.522]
BILL_AMT1
[0.742, 0.097, 0.256]
BILL_AMT2
[0.745, 0.082, 0.246]
BILL_AMT3
[0.743, 0.076, 0.227]
BILL_AMT4
[0.744, 0.073, 0.229]
BILL_AMT5
[0.748, 0.078, 0.25]
BILL_AMT6
[0.75, 0.082, 0.264]
PAY_AMT1
[0.765, 0.036, 0.237]
PAY_AMT2
[0.744, 0.051, 0.183]
PAY_AMT3
[0.764, 0.039, 0.242]
PAY_AMT4
[0.755, 0.044, 0.206]
PAY_AMT5
[0.763, 0.034, 0.218]
PAY_AMT6
[0.648, 0.412, 0.286]
